# Исследование музыкальных предпочтений пользователей больших городов

# Описание проекта

- Заказчик проекта — Яндекс.Музыка

- Задача проекта — сравнить музыкальные композиции, которые пользователи Москвы и Санкт-Петербурга прослушивают в понедельник утром, в среду, и в пятницу

# Описание данных

Данные выданы в csv-файле. Датасет включает следующие поля:

- 'userID' — уникальный идентификатор пользователя;
- 'Track' — название проигрываемой музыкальной композиции;
- 'artist' — имя исполнителя/название музыкальной группы;
- 'genre' —  жанр музыки;
- 'City' — город прослушивания композиции: Москва или Санкт-Петербург;
- 'time' — время, в которое пользователь прослушивал композицию;
- 'Day' — день недели.

# Содержание проекта

1. [Загрузка датасета и подготовка его к анализу](#1)


2. [Предобработка данных](#2)

    2.1. [Изучение столбцов таблицы](#2_1)
    
    2.2. [Проверка на наличие пропусков](#2_2)
    
    2.3. [Проверка на наличие дубликатов](#2_3)
    
    2.4. [Предоработка данных: вывод](#2_4)
    
    
3. [Исследовательский анализ](#3)

    3.1. [Как слушают музыку в разных городах?](#3_1)
    
    3.2. [Утро понедельника и вечер пятницы — разная музыка или одна и та же?](#3_2)
    
    3.3. [Москва и Питер — две разные столицы, два разных направления в музыке.](#3_3)
    
    
4. [Результаты исследования](#4)

# 1. Загрузка датасета и подготовка его к анализу<a id="1"></a>

In [1]:
import pandas as pd

music_project = pd.read_csv('...')
music_project = music_project[['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day']]
music_project.head()

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


In [2]:
print('Размер датасета music_project:', music_project.shape)

Размер датасета music_project: (65079, 7)


In [3]:
music_project.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


In [4]:
music_project.describe()

,userID,Track,artist,genre,City,time,Day
count,65079,63848,57876,63881,65079,65079,65079
unique,41748,47245,43605,287,2,20392,3
top,A8AE9169,Intro,Sardor Rahimxon,pop,Moscow,21:51:22,Friday
freq,76,34,6,8850,45360,14,23149


- В таблице 7 столбцов, у всех столбцов тип данных object, требуется преобразование типов данных в столбце time.
- Выявлены пробелы в названиях столбцов таблицы - необходимо переименовать столбцы.
- Количество значений в столбцах различается. Это говорит о том, что в данных есть пропущенные значения, их следует обработать.
- Для проверки гипотез важны столбцы *time*, *Day(понедельник, среда, пятница)* и *City(Москва и Санкт-Петербург)*. Данные из столбца *genre (287 жанров)* позволят узнать самые популярные жанры.

# 2. Предобработка данных<a id="2"></a>

## 2.1. Изучение столбцов таблицы<a id="2_1"></a>

In [5]:
# Переименовываем столбцы, удаляя пробелы

music_project.set_axis(['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time', 'weekday'], 
                       axis = 'columns', inplace = True)
music_project.columns

Index(['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time',
       'weekday'],
      dtype='object')

## 2.2. Проверка на наличие пропусков<a id="2_2"></a>

In [6]:
# Проверим на наличие пропусков

print('Количество пропусков в таблице music_project:')
print()
print(music_project.isnull().sum())

Количество пропусков в таблице music_project:

user_id           0
track_name     1231
artist_name    7203
genre_name     1198
city              0
time              0
weekday           0
dtype: int64


Есть пропущенные значения в столбцах track_name и artist_name - заменим их на маркер "unknown"

In [7]:
# Заменяем пропущенные значения на строку 'unknown'
music_project['track_name'] = music_project['track_name'].fillna('unknown')
music_project['artist_name'] = music_project['artist_name'].fillna('unknown')

music_project.isnull().sum()

user_id           0
track_name        0
artist_name       0
genre_name     1198
city              0
time              0
weekday           0
dtype: int64

In [8]:
# Перед удалением пропущенных значений в genre_name, проверим, какой процент данных датасета они занимают

print('Пропущенные значения столбца genre_name занимают {:.2%} данных датасета, можно их удалить'.format(
    music_project['genre_name'].isnull().sum()/music_project['genre_name'].count()))

Пропущенные значения столбца genre_name занимают 1.88% данных датасета, можно их удалить


In [9]:
# Удаление пропущенных значений в столбце 'genre_name'
music_project.dropna(subset = ['genre_name'], inplace = True)
music_project.isnull().sum()

user_id        0
track_name     0
artist_name    0
genre_name     0
city           0
time           0
weekday        0
dtype: int64

## 2.3. Проверка на наличие дубликатов<a id="2_3"></a>

In [10]:
# Поиск дубликатов в таблице music_project
music_project.duplicated().sum()

3755

Необходимо обратиться по поводу дубликатов к сотруднику, предоставившему выгрузку данных, - они могли появиться вследствие сбоя в записи данных.

In [11]:
# Удаление всех дубликатов из таблицы music_project
music_project = music_project.drop_duplicates().reset_index(drop = True)
music_project.duplicated().sum()

0

Сохраняем список уникальных значений столбца с жанрами в переменной genres_list.

Объявим функцию find_genre() для поиска неявных дубликатов в столбце с жанрами. Например, когда название одного и того же жанра написано разными словами.

In [12]:
# Сохранение в переменной genres_list списка уникальных значений

genres_list = music_project['genre_name'].unique()

# Создание функции find_genre()
def find_genre(genre):
    count = 0
    for i in genres_list:
        if i == genre:
            count += 1
    return count

Использование функции find_genre() для поиска вариантов названия жанра хип-хоп в таблице.

- hip
- hop
- hip-hop

In [13]:
# вызовом функции find_genre() проверяется наличие варианта 'hip'
find_genre('hip')

1

In [14]:
# проверяется наличие варианта 'hop'
find_genre('hop')

0

In [15]:
# проверяется наличие варианта 'hip-hop'
find_genre('hip-hop')

0

Создадим функцию find_hip_hop(), которая заменяет неправильное название в столбце 'genre_name' на 'hiphop'

In [16]:
# Создание функции find_hip_hop()
def find_hip_hop(music_project, wrong):
    music_project['genre_name'] = music_project['genre_name'].replace(wrong, 'hiphop')
    return music_project[music_project['genre_name'] == wrong]['genre_name'].count()

find_hip_hop(music_project,'hip')

0

In [17]:
# получение общей информации о данных таблицы
music_project.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60126 entries, 0 to 60125
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      60126 non-null  object
 1   track_name   60126 non-null  object
 2   artist_name  60126 non-null  object
 3   genre_name   60126 non-null  object
 4   city         60126 non-null  object
 5   time         60126 non-null  object
 6   weekday      60126 non-null  object
dtypes: object(7)
memory usage: 3.2+ MB


## 2.4. Предоработка данных: вывод<a id="2_4"></a>
На этапе предобработки данных были обнаружены проблемы с названиями столбцов (пробелы в названиях), пропущенные значения и дубликаты. 

Часть пропущенных значений была заменена на маркер "unknown", часть удалена, дубликаты были полностью удалены. 
Предобработка данных позволит провести более точный дальнейший анализ. 

# 3. Исследовательский анализ<a id="3"></a>

## 3.1. Как слушают музыку в разных городах?<a id="3_1"></a>

Выдвинута гипотеза: в Москве и Санкт-Петербурге пользователи слушают музыку по-разному. 

Необходимо проверить это предположение, сравнив, сколько времени включена музыка у жителей этих городов по понедельникам, средам и пятницам.

In [18]:
# Группируем данные по городу и подсчитываем композиции, для которых известен жанр
music_project.groupby('city')['genre_name'].count()

city
Moscow              41892
Saint-Petersburg    18234
Name: genre_name, dtype: int64

В Москве прослушиваний более чем в 2 раза больше, чем в Питере. 

Однако это не означает, что аудитория пользователей из Москвы более активна - в Москве в принципе больше пользователей Яндекс.Музыки, чем в Санкт-Петербурге.

In [19]:
# Сгруппируем данные по дню недели и подсчитаем прослушанные композиции, для которых известен жанр
music_project.groupby('weekday')['genre_name'].count()

weekday
Friday       21482
Monday       20866
Wednesday    17778
Name: genre_name, dtype: int64

В пятницу и понедельник пользователи более активно слушают музыку, чем в среду.

Создаём функцию number_tracks(), которая принимает как параметры таблицу, день недели и название города, а возвращает количество прослушанных композиций, для которых известен жанр. Проверяем количество прослушанных композиций для каждого города и понедельника, затем среды и пятницы.

In [20]:
# Создание функции number_tracks()
def number_tracks(music_project, day, city):
    track_list = music_project[(music_project['weekday'] == day) & (music_project['city'] == city)]
    track_list_count = track_list['genre_name'].count()
    return track_list_count

In [21]:
# список композиций для Москвы в понедельник
number_tracks(music_project, 'Monday', 'Moscow')

15347

In [22]:
# список композиций для Санкт-Петербурга в понедельник
number_tracks(music_project, 'Monday', 'Saint-Petersburg')

5519

In [23]:
# список композиций для Москвы в среду
number_tracks(music_project, 'Wednesday', 'Moscow')

10865

In [24]:
# список композиций для Санкт-Петербурга в среду
number_tracks(music_project, 'Wednesday', 'Saint-Petersburg')

6913

In [25]:
# <список композиций для Москвы в пятницу>
number_tracks(music_project, 'Friday', 'Moscow')

15680

In [26]:
# список композиций для Санкт-Петербурга в пятницу
number_tracks(music_project, 'Friday', 'Saint-Petersburg')

5802

Сведём полученную информацию в одну таблицу, где ['city', 'monday', 'wednesday', 'friday'] названия столбцов.

In [27]:
# таблица с полученными данными
data = [['Moscow', 15347, 10865, 15680],
       ['Saint-Petersburg', 5519, 6913, 5802]]
table = pd.DataFrame(data = data, columns = ['city','monday', 'wednesday', 'friday'])
table

,city,monday,wednesday,friday
0,Moscow,15347,10865,15680
1,Saint-Petersburg,5519,6913,5802


- В Москве больше всего музыку слушают в пятницу, чуть меньше в понедельник, и на приблизительно 30% меньше в среду
- В Санкт-Петерубурге больше всего слушают музыку по средам, почти на 20% меньше в пятницу и понедельник
- Можно сказать, что музыку слушают зеркально - в Москве пики активности пользователей приходятся на понедельник и среду, а в Санкт-Петербурге - на среду.

## 3.2. Утро понедельника и вечер пятницы — разная музыка или одна и та же?<a id="3_2"></a>

Ищем ответ на вопрос, какие жанры преобладают в разных городах в понедельник утром и в пятницу вечером. Есть предположение, что в понедельник утром пользователи слушают больше бодрящей музыки (например, жанра поп), а вечером пятницы — больше танцевальных (например, электронику).

In [28]:
# Получим таблицы данных по Москве moscow_general
moscow_general = music_project[music_project['city'] == 'Moscow']

In [29]:
# Получим таблицы данных по Санкт-Петербургу spb_general
spb_general = music_project[music_project['city'] == 'Saint-Petersburg']

Создаём функцию genre_weekday(), которая возвращает список жанров по запрошенному дню недели и времени суток с такого-то часа по такой-то.

In [30]:
# объявление функции genre_weekday() с параметрами music_project, day, time1, time2
def genre_weekday(music_project,day,time1,time2):
    genre_list = music_project[(music_project['weekday'] == day) & (music_project['time'] > time1 ) & (music_project['time'] < time2)]
    genre_list_sorted = genre_list.groupby('genre_name')['genre_name'].count().sort_values(ascending = False).head(10)
    return genre_list_sorted

Cравниваем полученные результаты по таблице для Москвы и Санкт-Петербурга в понедельник утром (с 7 до 11) и в пятницу вечером (с 17 до 23).

In [31]:
# вызов функции для утра понедельника в Москве
genre_weekday(moscow_general,'Monday','07:00:00','11:00:00')

genre_name
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
classical      157
Name: genre_name, dtype: int64

In [32]:
# вызов функции для утра понедельника в Петербурге>
genre_weekday(spb_general, 'Monday','07:00:00', '11:00:00')

genre_name
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre_name, dtype: int64

In [33]:
# вызов функции для вечера пятницы в Москве
genre_weekday(moscow_general,'Friday','17:00:00','23:00:00')

genre_name
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre_name, dtype: int64

In [34]:
# вызов функции для вечера пятницы в Питере
genre_weekday(spb_general, 'Friday','17:00:00', '23:00:00')

genre_name
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre_name, dtype: int64

В понедельник утром в двух городах популярные жанры похожи: больше всего слушают pop, rock, electronic, dance и hiphop. Наиболее популярен pop. Однако жанры топ-6 - топ10 уже отличаются: в Питере в топ-10 входит джаз, а в Москве жанр world.

В пятницу ситуация для топ-5 жанров не меняется: жанр pop - на первом месте. Опять разница заметна только в концовке топ-10, где в Москве пятничным вечером присутствует жанр ruspop, а в Питере появляется жанр world.

## 3.3. Москва и Питер — две разные столицы, два разных направления в музыке.<a id="3_3"></a>

Гипотеза: Питер богат своей рэп-культурой, поэтому это направление там слушают чаще, а Москва — город контрастов, но основная масса пользователей слушает попсу.

In [35]:
# Сгруппируем таблицу по жанру, сосчитаем численность композиций каждого жанра
moscow_genres = moscow_general.groupby('genre_name')['genre_name']
moscow_genres = moscow_genres.count().sort_values(ascending = False)

moscow_genres.head(10)

genre_name
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre_name, dtype: int64

In [36]:
# Сгруппируем таблицу spb_general по жанру, сосчитаем численность композиций каждого жанра
spb_genres = spb_general.groupby('genre_name')['genre_name'].count().sort_values(ascending = False)
spb_genres.head(10)

genre_name
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre_name, dtype: int64

В Москве, кроме абсолютно популярного жанра поп, есть направление русской популярной музыки. Значит, что интерес к этому жанру шире. А рэп, вопреки предположению, занимает в обоих городах близкие позиции.

# 4. Результаты исследования<a id="4"></a>
В процессе работы были выдвинуты следующие гипотезы:

- музыку в двух городах — Москве и Санкт-Петербурге — слушают в разном режиме;
- списки десяти самых популярных жанров утром в понедельник и вечером в пятницу имеют характерные отличия;
- население двух городов предпочитает разные музыкальные жанры.

Выводы:
- первая гипотеза была подтверждена - В Москве больше всего музыку слушают в понедельник и пятницу, а в Санкт-Петерубурге - по средам; 
- вторая гипотеза не была подтвержена - Москва и Петербург сходятся во вкусах: везде преобладает популярная музыка. При этом зависимости предпочтений от дня недели в каждом отдельном городе нет — люди постоянно слушают то, что им нравится. Но между городами в разрезе дней неделей наблюдается зеркальность относительно среды: Москва больше слушает в понедельник и пятницу, а Петербург наоборот - больше в среду, но меньше в понедельник и пятницу;
- третья гипотеза не была подтверждена - большинство популярных жанров из топ-10 в двух городах одинаковы